In [1]:
import random
import json
import numpy as np
import torch
import math
from nltk.tokenize import word_tokenize, sent_tokenize
import csv
import time
from tqdm import tqdm
import spacy

from data_loader import get_dataset_df
from run_sa import get_dataset
from rouge_score import rouge_scorer
from SA.args import get_model_args
from baselines import aggregate_print_rouges

import pandas as pd
import random

from nltk.parse.corenlp import CoreNLPParser

In [2]:
parser = CoreNLPParser('http://localhost:9000', encoding="utf-8")

## Dataset loader

In [3]:
sa_args = get_model_args()

# sa_args.dataset_name = 'liar'
# sa_args.sentences_path = "/Users/jolly/PycharmProjects/COPENLU/FilteredData/sup_sccores_liar/results_serialized_test_filtered.jsonl"
# sa_args.dataset_path = "/Users/jolly/PycharmProjects/COPENLU/liar_data/ruling_oracles_test.tsv"
# sa_args.top_n = 3

# dataset = get_dataset(sa_args)
# pipeline_inp = [line for line in open('/Users/jolly/PycharmProjects/COPENLU/data-used-for-HE-prep/liar/liar_sup_test.txt', 'r')] #sa_inp (pipeline-inp)+ '\t' +sa_out
# pipeline_out = [line for line in open('/Users/jolly/PycharmProjects/COPENLU/data-used-for-HE-prep/liar/liar_sup_test_pegasus.txt', 'r')]

sa_args.dataset_name = 'pubhealth'
sa_args.sentences_path = "/Users/jolly/PycharmProjects/COPENLU/data-used-for-HE-prep/pubhealth/results_serialized_test_filtered.jsonl"
sa_args.dataset_path = "/Users/jolly/PycharmProjects/COPENLU/data-used-for-HE-prep/pubhealth/test.tsv"
dataset = get_dataset(sa_args)
pipeline_inp = [line for line in open('/Users/jolly/PycharmProjects/COPENLU/data-used-for-HE-prep/pubhealth/pub_sup_test.txt', 'r')] #sa_inp (pipeline-inp)+ '\t' +sa_out
pipeline_out = [line for line in open('/Users/jolly/PycharmProjects/COPENLU/data-used-for-HE-prep/pubhealth/pub_sup_test_pegasus.txt', 'r')]
sa_args.top_n = 3


Namespace(C=300, algo_type='noSA', batch_size=1, dataset_name='pubhealth', dataset_path='/Users/jolly/PycharmProjects/COPENLU/liar_data/ruling_oracles_val.tsv', delete_th=0.97, device_type='cpu', editor_model_id='roberta-base', fluency_weight=1.4, fluencyscorer_model_id='gpt2', insert_th=1.1, length_weight=1.25, max_steps=200, min_length_of_edited_sent=40, named_entity_score_weight=0.95, outdir='', outfile='', outfile_filtered='', outfile_pegasus='', pegasus_modelname='tuner007/pegasus_paraphrase', reorder_th=0.95, sample=None, sbertname_pegasus='paraphrase-distilroberta-base-v1', seed=33, semantic_weight_keywords=1.0, semantic_weight_sentences=1.1, sentences_path='/Users/jolly/PycharmProjects/COPENLU/results_serialized_val_filtered.jsonl', split='test', t_init=60000, top_n=6)
Size of dataset: 1248
Sample:  {'claim_id': '11972.json', 'statement': 'Building a wall on the U.S.-Mexico border will take literally years.', 'justification': 'Perry said: "Building a wall" on the U.S.-Mexico bo

## Computing Rouge Scores

In [10]:
score_names = ['rouge1', 'rouge2', 'rougeLsum']
scorer = rouge_scorer.RougeScorer(score_names, use_stemmer=True)
scores = []

processed_samples = 0
for org in tqdm(dataset):
    processed_samples+=1
    #print(org['scored_sentences'])
    score1 = scorer.score(prediction='\n'.join(sent_tokenize(org['scored_sentences'])),
                              target='\n'.join(org['justification_sentences']))
    scores.append(score1)

    

print("Scores)")
aggregate_print_rouges(score_names, scores)
print("Processed: ", processed_samples)

100%|██████████| 1248/1248 [00:07<00:00, 178.06it/s]


Scores)
{'rouge1': AggregateScore(low=Score(precision=0.395032798626512, recall=0.3235236098291057, fmeasure=0.3291102821595914), mid=Score(precision=0.40441688548875565, recall=0.3311518163318101, fmeasure=0.33554521250787134), high=Score(precision=0.41431664625001186, recall=0.3384736012620341, fmeasure=0.34200122273969036)),
 'rouge2': AggregateScore(low=Score(precision=0.15555399988337748, recall=0.12075426635918562, fmeasure=0.12497195824768835), mid=Score(precision=0.16438397471201333, recall=0.12722761820315598, fmeasure=0.1311786919246722), high=Score(precision=0.17286865800918366, recall=0.13371411422600787, fmeasure=0.13743479698852543)),
 'rougeLsum': AggregateScore(low=Score(precision=0.34763133050842415, recall=0.28244710678224605, fmeasure=0.2881896767842884), mid=Score(precision=0.3571951596940012, recall=0.2890474783509336, fmeasure=0.29407225188022723), high=Score(precision=0.3667481324105058, recall=0.29626276989477757, fmeasure=0.3001459355001099))}
rouge1 P: 40.44 	

## Data prep for Task1

In [ ]:
random.seed(420)# - To save data for task 1
final_data = []

for org, inp, out in zip(dataset, pipeline_inp, pipeline_out):
    
    claim = org["statement"]
    veracity_label = org["label"]
    pipe_inp = inp.split("\t")[0]
    pipe_out = out
    
    line_data = {
        "claim": claim,
        "label": veracity_label,
        "pipe_inp": pipe_inp,
        "pipe_out": pipe_out}
    
    final_data.append(line_data)

final_data_40 = []
for idx, line in enumerate(random.sample(final_data, 40)):
    line["id"] = idx+1
    final_data_40.append(line)
    
json.dump(final_data_40, open("he_pub_task1.json", "w"), indent=2)
print(len(final_data_40))

## Data prep for Task2

In [ ]:
new_final_data = []

In [ ]:
#To save data for task2

random.seed(40) 

final_data = []

for org, inp, out in zip(dataset, pipeline_inp, pipeline_out):
    
    claim = org["statement"]
    veracity_label = org["label"]
    pipe_inp = inp.split("\t")[0]
    pipe_out = out
    
    line_data = {
        "claim": claim,
        "label": veracity_label,
        "just": pipe_inp,
        "just_type": "pipe_inp"
    }
    
    final_data.append(line_data)
for idx, line in enumerate(random.sample(final_data, 30)):
    new_final_data.append(line)
print(len(new_final_data))


In [ ]:
final_data_60 = []
for idx, line in enumerate(random.sample(new_final_data, 60)):
    line["id"] = idx+1
    final_data_60.append(line)
    
json.dump(final_data_60, open("he_pub_task2.json", "w"), indent=2)
print(len(final_data_60))

## Read SA output file

In [ ]:
pwd

In [ ]:
file1 = [line for line in open('../OUTs-Server/to_Pepa_forlabelpred/pub_sup/pub_sup_test.txt')]

file2 = [line for line in open('../OUTs-Server/to_Pepa_forlabelpred/pub_sup/pub_sup_test_pp.txt')]

file3 = [line for line in open('../OUTs-Server/to_Pepa_forlabelpred/pub_sup/pub_sup_test_pegasus.txt')]

for i, j, k, org in zip(file1, file2, file3, dataset):
    
    print(i.split("\t")[0] + "\n")
    print(j + "\n")
    print(k + "\n")
    print("claim: ", org["statement"])
    print("label: ", org["label"])
    input()
    

## CleanSents using Paraphrase Tools

In [ ]:
import torch
import language_tool_python

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from sentence_transformers import SentenceTransformer, util

tool = language_tool_python.LanguageTool('en-US')
nlp = spacy.load("en_core_web_sm")

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

model_sbert = SentenceTransformer('paraphrase-distilroberta-base-v1')

num_beams = 10
num_return_sequences = 10

def get_response(input_text,num_return_sequences,num_beams):
    batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch, max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

def gramatical_tool(sent):
    matches = tool.check(sent)
    return language_tool_python.utils.correct(sent, matches)

def sentence_level_semantic_scorer_sbert(org, rep):
    org_embeds = model_sbert.encode(org)
    rep_embeds = model_sbert.encode(rep)
    return torch.FloatTensor([util.pytorch_cos_sim(e1, e2) for e1, e2 in zip(org_embeds, rep_embeds)])

def post_process(text):
    valid_sents = []
    for i in sent_tokenize(text):
        i = gramatical_tool(i)
        doc = nlp(i)
        verbs = [token.text for token in doc if token.pos_ in ['VERB', 'AUX']]
        if len(verbs)>0:
            valid_sents.append(i)
            
    return " ".join(valid_sents)

def pegasus(text):
    temp = []
    for i in sent_tokenize(text):
        if len(i.split(" ")) == 1:
            temp.append(i)
            continue
        else:
            all_responses = get_response(i, num_return_sequences, num_beams)
            temp_str = ''
            sim = sentence_level_semantic_scorer_sbert(all_responses, [i]*10)
            max_sim_rep = all_responses[torch.argmax(sim)]    
            temp.append(max_sim_rep)
    
    return(" ".join(temp))
    


In [ ]:
outs = json.load(open("he_data_liar.json"))
for line in tqdm(outs):
    line['sa_pp'] = post_process(line["sa_out"])
    line['sa_pegasus'] = pegasus(line["sa_pp"])
    

In [ ]:
#json.dump(outs, open('new_postprocess_liar.json', 'w+'))
import json
outs = json.load(open("new_postprocess_liar.json"))
for line in outs:
    print("sa_inp", line["sa_inp"])
    print("-----------")
    print("sa_out", line["sa_out"])
    print("-----------")
    print("sa_pp", line["sa_pp"])
    print("-----------")
    print("sa_pegasus", line["sa_pegasus"])
    print("-----------")
    input()

In [ ]:
JustFact: Generating fact-checking explainations in unsupervised settings

Pipeline - Sentences from RC's --> SA --> post-processing to remove grammatical errors --> Pegasus to make it more consise

Human Eval - Using two justifications - pipeline inp (SA inp) & pipeline out (pegasus out)

## GPT-Debugging

In [ ]:
import torch
import math

from torch.nn import CrossEntropyLoss
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2").to("cpu").eval()

In [ ]:
def scorer_batch(sentences):
    #Gpt for fluency
    tokenizer.pad_token = tokenizer.eos_token
    tensor_input = {k: v.to("cpu") for k,v in tokenizer(sentences, padding=True, truncation=True, return_tensors='pt').items()}


    lm_labels = tensor_input["input_ids"].detach().clone()
    lm_labels[lm_labels[:, :] == tokenizer.pad_token_id] = -100

    outputs = model(input_ids=tensor_input["input_ids"],
                attention_mask= tensor_input["attention_mask"],
                return_dict=True)

    lm_logits = outputs.logits
    shift_logits = lm_logits[..., :-1, :].contiguous()
    shift_labels = lm_labels[..., 1:].contiguous()
    
    print([tokenizer._convert_id_to_token(i) for i in shift_labels.tolist()[0]])

    loss_fct = CrossEntropyLoss(ignore_index=-100, reduction='none')  # give CE loss at each word generation step
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    
    log_prob_sum = loss.reshape(-1, shift_labels.shape[-1]) #.sum(dim=1)
    log_prob_sum1 = torch.exp(-loss.reshape(-1, shift_labels.shape[-1])) #.sum(dim=1)
    len_sum = tensor_input["attention_mask"][..., 1:].contiguous() #.sum(dim=1)
    
    #prob_products_per_sample = torch.exp(-1 * (log_prob_sum/len_sum)).cpu()

    print(log_prob_sum)
    #print(log_prob_sum1)
    print(len_sum)
    
    print(log_prob_sum.sum(dim=1))
    #print(log_prob_sum1)
    print(len_sum.sum(dim=1))
    print(log_prob_sum.sum(dim=1) / len_sum.sum(dim=1))
    
    print("\nFinal:", 100 * torch.exp(- 1 * (log_prob_sum.sum(dim=1) / len_sum.sum(dim=1))))
    
    #return (prob_products_per_sample * 100)
    
sents = ['I bought bananas, apples, and orange juice from the supermarket.']
scorer_batch(sents)

## Data Prep SA input

In [ ]:
def clean_str(sent):
    sent = sent.replace("’", "'")
    sent = sent.replace("‘", "`")
    sent = sent.replace('"', "''")
    sent = sent.replace("—", "--")
    sent = sent.replace("…", "...")
    sent = sent.replace("–", "--")

    return sent

def get_dataset(scored_sentences_path, dataset_path, dataset_name, top_n, parser):

    if dataset_name == 'liar_plus':
        df = pd.read_csv(dataset_path, sep='\t', index_col=0)
        df = df.dropna()
        columns = ['dummy', 'id', 'statement', 'justification',
               'ruling_without_summary', 'label', 'just_tokenized',
               'ruling_tokenized', 'statement_tokenized', 'oracle_ids']
        print(df.columns)
        print(columns)
        df.columns = columns
        
    elif dataset_name == 'pub_health':
        df = pd.read_csv(dataset_path, sep='\t', index_col=0)
        df = df.dropna()
        
        columns = ['claim_id', 'claim', 'date_published', 'explanation',
                   'fact_checkers', 'main_text', 'sources', 'label', 'subjects']
        
        if len(df.columns) == 10:
            columns = ['dummy'] + columns
        
        df.columns = columns
        
    scored_sentences = [json.loads(line) for line in open(scored_sentences_path)]
    scored_sentences = {item["id"]: sorted(item['sentence_scores'], key=lambda x: x[1], reverse=True)[:top_n] for item in scored_sentences}
    
    
    inp_scored_sentences = {}
    for k, v in scored_sentences.items():
        
        temp = []
        for sent in v:
            temp.append(sent[0])
        inp_scored_sentences[k] = clean_str(" ".join(temp))

    scored_sentences = inp_scored_sentences
    
    
    if dataset_name == 'liar_plus':
        
        df['scored_sentences'] = df.apply(lambda x: scored_sentences.get(x['id'], None), axis=1)
        df = df[df['scored_sentences'] != None]
        df['justification_sentences'] = df.apply(lambda x: sent_tokenize(x['justification']), axis=1)
        df = df[['id', 'statement', 'justification', 'label', 'scored_sentences',
             'justification_sentences']]
        
    elif dataset_name == 'pub_health':
        df['claim_id'] = df['claim_id'].astype('str')
        df['scored_sentences'] = df.apply(lambda x: scored_sentences.get(x['claim_id'], None), axis=1)
        df = df[df['scored_sentences'] != None]
        df['justification_sentences'] = df.apply(lambda x: sent_tokenize(x['explanation']), axis=1)
        df = df[['claim_id', 'claim', 'explanation', 'label', 'scored_sentences',
             'justification_sentences']]
        
        
    dataset = [row.to_dict() for i, row in df.iterrows()]
    new_dataset = []
    if dataset_name == 'liar_plus':
        for i in dataset:
            if i["scored_sentences"] is None or i["id"] == '2001.json': #Sentence in Liarplus is too long:
                continue
            else:
                new_dataset.append(i)
    elif dataset_name == 'pub_health':
        for i in dataset:
        
            if i["scored_sentences"] is None or i["scored_sentences"] == None:
                continue
            else:
                new_dataset.append(i)
    

    print(f'Size of dataset: {len(dataset)}')
    print(f'Size of new dataset: {len(new_dataset)}')
    print('Sample: ', dataset[0])
    if len(new_dataset)!=0:
        print('Sample: ', new_dataset[0])

    return 

scored_sentences_path = "../../DATA-COPE-Project-DIKUServer/unsup_scores_liar/sentence_scores_val.jsonl" #Each line is a json
scored_sentences_path1 = "../../DATA-COPE-Project-DIKUServer/unsup_scores_pubhealth/sentence_scores_test.jsonl"

dataset_path = "../../liar_data/ruling_oracles_val.tsv"
dataset_path1 = "../../DATA-COPE-Project-DIKUServer/PUBHEALTH/test.tsv"

get_dataset(scored_sentences_path1, dataset_path1, 'pub_health', 6, parser)
#get_dataset(scored_sentences_path, dataset_path, 'liar_plus', 6, parser)
